In [1]:
# will run in multi gpu

In [2]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import argparse
import random
import numpy as np
from HyerIQASolver import HyperIQASolver

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
import data_path

In [6]:
def main(config):

    folder_path = {
        'live': '/home/ssl/Database/databaserelease2/',
        'csiq': '/home/ssl/Database/CSIQ/',
        'tid2013': '/home/ssl/Database/TID2013/',
        # 'livec': '/home/ssl/Database/ChallengeDB_release/ChallengeDB_release/',
        'livec': data_path.clive,
        # 'koniq-10k': '/home/ssl/Database/koniq-10k/',
        'koniq-10k': data_path.koniq10k,
        'bid': '/home/ssl/Database/BID/',
    }

    img_num = {
        'live': list(range(0, 29)),
        'csiq': list(range(0, 30)),
        'tid2013': list(range(0, 25)),
        'livec': list(range(0, 1162)),
        # 'koniq-10k': list(range(0, 10073)),
        'koniq-10k': list(range(0, 1200)),
        'bid': list(range(0, 586)),
    }
    
    config = config[0] # tuple issue fix
    sel_num = img_num[config.dataset]

    # srcc_all = np.zeros(config.train_test_num, dtype=np.float)
    # plcc_all = np.zeros(config.train_test_num, dtype=np.float)
    srcc_all = np.zeros(config.train_test_num, dtype=float)
    plcc_all = np.zeros(config.train_test_num, dtype=float)

    print('Training and testing on %s dataset for %d rounds...' % (config.dataset, config.train_test_num))
    for i in range(config.train_test_num):
        print('Round %d' % (i+1))
        # Randomly select 80% images for training and the rest for testing
        random.shuffle(sel_num)
        train_index = sel_num[0:int(round(0.8 * len(sel_num)))]
        test_index = sel_num[int(round(0.8 * len(sel_num))):len(sel_num)]

        solver = HyperIQASolver(config, folder_path[config.dataset], train_index, test_index)
        srcc_all[i], plcc_all[i] = solver.train()

    # print(srcc_all)
    # print(plcc_all)
    srcc_med = np.median(srcc_all)
    plcc_med = np.median(plcc_all)

    print('Testing median SRCC %4.4f,\tmedian PLCC %4.4f' % (srcc_med, plcc_med))

    # return srcc_med, plcc_med

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', dest='dataset', type=str, default='koniq-10k', help='Support datasets: livec|koniq-10k|bid|live|csiq|tid2013')
    parser.add_argument('--train_patch_num', dest='train_patch_num', type=int, default=25, help='Number of sample patches from training image')
    parser.add_argument('--test_patch_num', dest='test_patch_num', type=int, default=25, help='Number of sample patches from testing image')
    parser.add_argument('--lr', dest='lr', type=float, default=2e-5, help='Learning rate')
    parser.add_argument('--weight_decay', dest='weight_decay', type=float, default=5e-4, help='Weight decay')
    parser.add_argument('--lr_ratio', dest='lr_ratio', type=int, default=10, help='Learning rate ratio for hyper network')
    parser.add_argument('--batch_size', dest='batch_size', type=int, default=96, help='Batch size')
    
    # parser.add_argument('--epochs', dest='epochs', type=int, default=16, help='Epochs for training')
    parser.add_argument('--epochs', dest='epochs', type=int, default=3, help='Epochs for training')
    
    parser.add_argument('--patch_size', dest='patch_size', type=int, default=224, help='Crop size for training & testing image patches')
    
    # parser.add_argument('--train_test_num', dest='train_test_num', type=int, default=10, help='Train-test times')
    parser.add_argument('--train_test_num', dest='train_test_num', type=int, default=3, help='Train-test times')

    # config = parser.parse_args()
    config = parser.parse_known_args()
    # print(config)
    main(config)

Training and testing on koniq-10k dataset for 3 rounds...
Round 1
Epoch	Train_Loss	Train_SRCC	Test_SRCC	Test_PLCC


/home/ec2-user/SageMaker/AmazonSageMaker-AdsQuality/code/HyperIQ/HyerIQASolver.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img.cuda())
/home/ec2-user/SageMaker/AmazonSageMaker-AdsQuality/code/HyperIQ/HyerIQASolver.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label.cuda())
/home/ec2-user/SageMaker/AmazonSageMaker-AdsQuality/code/HyperIQ/HyerIQASolver.py:105: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img.cuda())
/home/ec2-user/SageMaker/AmazonSageMaker-AdsQualit

1	14.228		0.5454		0.8804		0.9098
2	4.501		0.9232		0.8885		0.9099
3	3.999		0.9404		0.8885		0.9151
Best test SRCC 0.888543, PLCC 0.909913
Round 2
Epoch	Train_Loss	Train_SRCC	Test_SRCC	Test_PLCC
1	14.997		0.5226		0.8750		0.8859
2	4.549		0.9229		0.8827		0.8959
3	4.064		0.9394		0.8926		0.9045
Best test SRCC 0.892576, PLCC 0.904530
Round 3
Epoch	Train_Loss	Train_SRCC	Test_SRCC	Test_PLCC
1	14.335		0.5466		0.8355		0.8432
